In [110]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [117]:
import os
import re
import numpy as np
import pandas as pd
import re

from tensorflow.keras.layers import Embedding

2023-09-16 10:32:01.706966: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 10:32:01.858752: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 10:32:01.859498: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-16 10:32:03.795056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [102]:
WORD_SPACE_DIMENSION = 50

embeddings_index = {}
with open('/home/tina/Downloads/glove.6B.50d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [91]:
def get_distances(v1, v2):
    euclid_dist = distance.euclidean(v1,v2)
    cosine_similarity = np.dot(v1,v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
    return ('euclid_dist', euclid_dist), ('cosine_similarity', cosine_similarity), ('angle', np.arccos(cosine_similarity))

In [67]:
def compare_words():
    words = [
        ['man', 'woman'],
        ['girl', 'boy'],
        ['tiger', 'lion'],
        ['book', 'car'],
        ['computer', 'doll'],
    ]
    for w in words:
        print(w, '=>', get_distances(embeddings_index[w[0]], embeddings_index[w[1]]))

In [65]:
compare_words()

['man', 'woman'] => (('euclid_dist', 2.602623701095581), ('cosine_similarity', 0.8860338))
['girl', 'boy'] => (('euclid_dist', 2.042633295059204), ('cosine_similarity', 0.9327199))
['tiger', 'lion'] => (('euclid_dist', 4.520913124084473), ('cosine_similarity', 0.54939914))
['book', 'car'] => (('euclid_dist', 6.505012512207031), ('cosine_similarity', 0.29437548))
['computer', 'doll'] => (('euclid_dist', 6.290210247039795), ('cosine_similarity', 0.30855522))


In [235]:
MAX_TEXT_LEN = 200

def text2vec(text):
    if len(text.split()) < 3:
        return None
        
    pattern = r'[^a-zA-Z\s]'
    clean_text = re.sub(pattern, '', text)
    
    vec = []
    for w in clean_text.split():
        wv = embeddings_index.get(w)
        if wv is not None and len(vec) < MAX_TEXT_LEN:
           vec.append(wv)     
    
    if len(vec) == 0:
        return None
    
    pad_len = MAX_TEXT_LEN-len(vec)      
    if pad_len > 0:
        padding = [np.zeros_like(vec[0])] * pad_len
        vec = vec + padding
    return np.array(vec)

In [236]:
text1 = 'i have a cats'
text11 = 'radio wastes our time but not dog'
text12 = 'i like animals'

v1 = text2vec(text1)
v11 = text2vec(text11)
v12 = text2vec(text12)

v1 = v1.reshape(MAX_TEXT_LEN*WORD_SPACE_DIMENSION,)
v11 = v11.reshape(MAX_TEXT_LEN*WORD_SPACE_DIMENSION,)
v12 = v12.reshape(MAX_TEXT_LEN*WORD_SPACE_DIMENSION,)

print((text1, text11), '\n =>', get_distances(v1,v11), '\n')
print((text1, text12), '\n =>', get_distances(v1,v12), '\n')

('i have a cats', 'radio wastes our time but not dog') 
 => (('euclid_dist', 14.611369132995605), ('cosine_similarity', 0.30324912), ('angle', 1.2626958)) 

('i have a cats', 'i like animals') 
 => (('euclid_dist', 8.260468482971191), ('cosine_similarity', 0.68363035), ('angle', 0.81807095)) 



In [238]:
df = pd.read_csv('/home/tina/Downloads/questions.csv', usecols=['Question'])
df = df.dropna()
questions = df.Question.unique()

In [253]:
def find_closest(text):
    min_distance = None
    min_text = None
    v1 = text2vec(text)
    v1 = v1.reshape(MAX_TEXT_LEN*WORD_SPACE_DIMENSION,)
    for q in questions:
        v2 = text2vec(q)
        if v2 is None:
            continue
        
        v2 = v2.reshape(MAX_TEXT_LEN*WORD_SPACE_DIMENSION,)
        # dist = distance.euclidean(v1,v2)
        dist = 2 - np.dot(v1,v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))        
        if min_distance is None:
            min_distance = dist
            min_text = q
        elif min_distance > dist:
            min_distance = dist
            min_text = q
    
    return min_text

In [258]:
text = "how many types of trumpet are there?"

find_closest(text)

'Are there several types of trumpet?'